### import libraries

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

### get wikipedia html

In [2]:
wikipediaURL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
resp = requests.get(wikipediaURL)
soup = BeautifulSoup(resp.content, 'lxml')

### parse html table

In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        continue
    cell['PostalCode'] = row.p.contents[0]
    info = (row.p.contents[2].text.split('('))
    cell['Borough'] = info[0]
    cell['Neighborhood'] = info[1].strip(')').replace(' /',',').replace(')',' ').strip(' ')
    # cell['Borough'] = (row.span.text).split('(')[0]
    # cell['Neighborhood']
    # for i in row.p:
    #     print(i)
    table_contents.append(cell)

### create dataframe from the table

In [4]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


# ANSWER TO FIRST PART:

In [5]:
df.shape

(103, 3)

### Import geospatial data:

In [18]:
df_geodata = pd.read_csv('Geospatial_Coordinates.csv')

### Merge both dataframes into one

# ANSWER TO SECOND PART

In [31]:
df_fulldata = pd.merge(df, df_geodata, left_on='PostalCode', right_on='Postal Code')
df_fulldata.drop(columns=['Postal Code'], inplace=True)
df_fulldata.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
